In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ViT(nn.Module):
    def __init__(self, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, dropout_rate, channels=3):
        super().__init__()
        self.patch_size = patch_size

        # 이미지는 패치로 분할되고, 각 패치는 Transformer에 입력될 수 있도록 임베딩되어야 합니다.
        num_patches = (image_size // patch_size) ** 2
        patch_dim = channels * patch_size ** 2
        self.patch_to_embedding = nn.Linear(patch_dim, dim)

        # 클래스 토큰을 추가합니다. 이 토큰은 분류를 위해 사용됩니다.
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))

        # 위치 임베딩은 Transformer 모델에 시퀀스의 순서 정보를 제공합니다.
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))

        # Transformer 인코더를 정의합니다.
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, dropout=dropout_rate, activation='gelu'),
            num_layers=depth
        )

        # 분류를 위한 MLP 헤드입니다.
        self.mlp_head = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(mlp_dim, num_classes)
        )

    def forward(self, x):
        b, c, h, w = x.shape

        # 이미지를 패치로 분할하고 임베딩합니다.
        x = x.reshape(b, c, h // self.patch_size, self.patch_size, w // self.patch_size, self.patch_size)
        x = x.transpose(2, 4).flatten(2)
        x = self.patch_to_embedding(x)

        # 클래스 토큰과 위치 임베딩을 추가합니다.
        cls_tokens = self.cls_token.expand(b, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding

        # Transformer를 통과시킵니다.
        x = self.transformer(x)

        # 분류를 위해 첫 번째 토큰 (클래스 토큰)만 사용합니다.
        x = x[:, 0]

        return self.mlp_head(x)

# 예시 사용
vit = ViT(image_size=256, patch_size=32, num_classes=10, dim=1024, depth=6, heads=8, mlp_dim=2048, dropout_rate=0.1)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
